In [2]:
import requests
from bs4 import BeautifulSoup

In [34]:
url_base = "http://www.discosdobrasil.com.br/discosdobrasil/consulta/consulta.php?IndiceFixo=&Conteudo=Disco&Procura=&FormularioAnterior=&Apartirde=0&NumeroItens=7325&Formulario=Exibir"

In [44]:
def get_links_discos(url_base):
    source = requests.get(url_base)
    soup = BeautifulSoup(source.content,  "html.parser")
    links_discos = ['http://www.discosdobrasil.com.br/discosdobrasil/consulta/' + \
                    a['href'] for a in soup.find_all('a', href=True) \
                    if 'Id_Disco' in a['href']]
    return list(set(links_discos))

In [67]:
def get_albumName(soup):
    album = soup.find('p', attrs={'class':"destacadorealcadorecuado"})
    return album.text

In [107]:
def get_interpretes(soup):
    interpretes = [a.text.strip() for a in soup.find('p', 
             attrs={'class': 'realcadorecuado'}).findChildren("a", recursive=True)]
    return interpretes

In [176]:
def get_infosDisco(soup):
    table = soup.find_all('table', attrs={'width': '460'})[2]
    
    chaves = [td.find("p", recursive=False).text.replace(':', '') 
              for td in table.find_all('td', attrs={'width': '25%'})]
    
    valores = [td.find("p", recursive=False).text.strip() 
           for td in table.find_all('td', attrs={'width': '75%'})]
    
    infos_disco = {chave:valor for chave, valor in zip(chaves, valores)}
    
    return infos_disco

In [255]:
def rangeFaixas(soup):
    tables = soup.find_all('table', {'width':"460",
                                 'border':"0",
                                 'align':"center",
                                 'cellpadding':"0",
                                 'cellspacing':"0",
                                 'bgcolor':"#FAF7D6"})
    indices = []
    for i in range(len(tables)):
        p = tables[i].find('p', {'class':"destacadorecuado"})
        if p != None and 'faixa' in p.text:
            indices.append(i)
        
    range_faixas = []
    n = indices[-1]
    for i in range(len(indices)):
        faixa = []
        if indices[i] < n:
            ind1, ind2 = indices[i], indices[i+1]
            faixa = tables[ind1:ind2]
            range_faixas.append(faixa)
        else:
            faixa = tables[n:]
            range_faixas.append(faixa)
    return range_faixas

In [294]:
def get_infoFaixa(range_faixa):
    musicos = []
    compositores = []
    for table in range_faixa:
        a_ = table.find_all('a')
        p_ = table.find('p', {'class':"destacadorecuado"})
        for a in a_:
            if a != None and 'Id_Musica' in a['href']:
                musica = a.text
            if a != None and 'Id_Musico' in a['href']:
                musico_instrumento = a.parent.text
                musicos.append(musico_instrumento)
            if a != None and 'Id_Compositor' in a['href']:
                compositor = a.text
                compositores.append(compositor)
        if p != None and 'faixa' in p.text:
            faixa = p.text
            
    dic = {'musica': musica,
          'compositores': compositores,
          'musicos': musicos}
    return faixa, dic

In [45]:
links_discos = get_links_discos(url_base)

In [46]:
len(links_discos)

7325

In [47]:
links_discos

['http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI04994',
 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI00516',
 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI00699',
 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI02508',
 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI04648',
 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI05575',
 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI00434',
 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI05525',
 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI06514',
 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI00138',
 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI07102',
 'http://w

In [115]:
exe1 = "http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI05284"
ex2 = "http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI03054"
ex3 = "http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI03231"
ex4 = 'http://www.discosdobrasil.com.br/discosdobrasil/consulta/detalhe.php?Id_Disco=DI07317'
exemplos = [exe1, ex2, ex3, ex4]

<table width="460" border="0" align="center" cellpadding="0" cellspacing="0" bgcolor="#FAF7D6">
        <tbody><tr>
            <td align="right" valign="center" width="20%">
                <p class="destacadorecuado">faixa 01</p>
            </td>
            <td valign="center" width="10%">
                <a href="../musicas/DI05284MU038761.WMA" onfocus="if(this.blur)this.blur()" onmouseout="MM_swapImgRestore()" onmouseover="MM_swapImage('ouvir','','../figuras/bt_ouvir_b.gif',1)"><img src="../figuras/bt_ouvir_a.gif" alt="Clique para ouvir a música" name="ouvir" width="30" height="30" border="0"></a>
            </td>
            <td valign="center" width="70%"><p class="realcadominirecuado"><a href="detalhe.php?Id_Musica=MU038761" onmouseover="ddrivetip(' Exibe a relação dos discos com esta música.')" ;="" onmouseout="hideddrivetip()" onfocus="if(this.blur)this.blur()" onclick="AbreJanela(this.href,'Detalhe83650','517','600','yes');return false;">O Atirador </a></p></td>
        </tr>
        <tr>
            <td valign="top" width="20%">
                &nbsp;
            </td>
            <td valign="top" width="10%">
                &nbsp;
            </td>
            <td valign="top" width="70%">
                &nbsp;
            </td>
        </tr>
    </tbody></table>

In [215]:
r = requests.get(exemplos[0])
soup = BeautifulSoup(r.content, 'html.parser')

In [216]:
faixas = [p.text for p in soup.find_all('p', {'class':"destacadorecuado"}) if 'faixa' in p.text]
faixas

['faixa 01',
 'faixa 02',
 'faixa 03',
 'faixa 04',
 'faixa 05',
 'faixa 06',
 'faixa 07',
 'faixa 08',
 'faixa 09',
 'faixa 10',
 'faixa 11',
 'faixa 12',
 'faixa 13',
 'faixa 14']

In [217]:
musicas = [a.text.strip() for a in soup.find_all('a') if 'Id_Musica' in a['href']]
musicas

['O Atirador',
 'Hoje Eu Quero Sair Só',
 'A Rede',
 'Paciência',
 'O Último Pôr Do Sol',
 'O Homem Dos Olhos De Raio X',
 'Lá E Cá',
 'A Medida Da Paixão',
 'Tudo Por Acaso',
 'Miedo',
 'Santana',
 'A Ponte',
 'Dois Olhos Negros',
 'Jack Soul Brasileiro']

<table width="460" border="0" align="center" cellpadding="0" cellspacing="0" bgcolor="#FAF7D6">
        <tbody><tr>
            <td align="right" valign="center" width="20%">
                <p class="destacadorecuado">faixa 14</p>
            </td>
            <td valign="center" width="10%">
                <a href="../musicas/DI05284MU015254.WMA" onfocus="if(this.blur)this.blur()" onmouseout="MM_swapImgRestore()" onmouseover="MM_swapImage('ouvir','','../figuras/bt_ouvir_b.gif',1)"><img src="../figuras/bt_ouvir_a.gif" alt="Clique para ouvir a música" name="ouvir" width="30" height="30" border="0"></a>
            </td>
            <td valign="center" width="70%"><p class="realcadominirecuado"><a href="detalhe.php?Id_Musica=MU015254" onmouseover="ddrivetip(' Exibe a relação dos discos com esta música.')" ;="" onmouseout="hideddrivetip()" onfocus="if(this.blur)this.blur()" onclick="AbreJanela(this.href,'Detalhe81964','517','600','yes');return false;">Jack Soul Brasileiro </a></p></td>
        </tr>
        <tr>
            <td valign="top" width="20%">
                &nbsp;
            </td>
            <td valign="top" width="10%">
                &nbsp;
            </td>
            <td valign="top" width="70%">
                &nbsp;
            </td>
        </tr>
    </tbody></table>

In [256]:
faixas = rangeFaixas(soup)

In [286]:
range_faixa = faixas[1]

In [295]:
get_infoFaixa(range_faixa)

('faixa 02',
 {'compositores': ['Caxa Aragão ', 'Lenine ', 'Mu Chebadi '],
  'musica': 'Hoje Eu Quero Sair Só ',
  'musicos': ['Guila : Baixo Semi-Acústico',
   'Jr. Tostoi : Craviola 12 Cordas',
   'Lenine : Violão',
   'Pantico Rocha : Bateria']})

In [296]:
get_albumName(soup)

'ACÚSTICO MTV - LENINE                                                                                                   '

In [297]:
get_interpretes(soup)

['Lenine']

In [298]:
get_infosDisco(soup)

{'Característica': 'vocal',
 'Formatos': '(CD/2006)',
 'Gravadora': 'Sony&BMG;',
 'Observação': '',
 'Primeiro disco': '2006',
 'Produtor': 'Lenine/Alvaro Alencar/Leninha Brandão'}